In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.pipeline import Pipeline
import sklearn.model_selection
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [34]:
df = pd.read_csv(r"C:\Users\Awan\Desktop\car data.csv")

In [35]:
df.head()

,Car_Name,Year,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner
0,Toyota Corolla,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,Toyota Camry,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,Toyota Yaris,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,Toyota Prius,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,Toyota Fortuner,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [36]:
df.columns

Index(['Car_Name', 'Year', 'Selling_Price', 'Present_Price', 'Driven_kms',
       'Fuel_Type', 'Selling_type', 'Transmission', 'Owner'],
      dtype='object')

In [37]:
df.isnull().sum()


Car_Name         0
Year             0
Selling_Price    0
Present_Price    0
Driven_kms       0
Fuel_Type        0
Selling_type     0
Transmission     0
Owner            0
dtype: int64

In [ ]:
df = pd.get_dummies(df, drop_first=True)

In [71]:
X = df.drop('Selling_Price', axis=1)
y = df['Selling_Price']

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
model = Pipeline([("scaler", StandardScaler()),("regressor", LinearRegression())])

In [49]:

lr = LinearRegression()
rf = RandomForestRegressor(n_estimators=100, random_state=42)

In [50]:
lr.fit(X_train, y_train)
rf.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [56]:
lr_pred = lr.predict(X_test)
rf_pred = rf.predict(X_test)

In [58]:

model = RandomForestRegressor()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [53]:
print("Linear Regression R2:", r2_score(y_test, lr_pred))


Linear Regression R2: 0.8716291936548771


In [54]:
print("Random Forest R2:", r2_score(y_test, rf_pred))

Random Forest R2: 0.9615411547705366


In [55]:
import joblib


In [60]:
joblib.dump(rf, "car_price_model.pkl")
joblib.dump(X.columns.tolist(), "car_features.pkl")

['car_features.pkl']

In [61]:
print("Features saved:", X.columns.tolist())

Features saved: ['Year', 'Present_Price', 'Driven_kms', 'Owner', 'Car_Name_Audi A Four', 'Car_Name_Audi A One', 'Car_Name_Audi A Six', 'Car_Name_Audi A Three', 'Car_Name_Audi Q Eight', 'Car_Name_Audi Q Five', 'Car_Name_Audi Q Seven', 'Car_Name_Audi Q Three', 'Car_Name_Audi Q Two', 'Car_Name_BMW Five Series', 'Car_Name_BMW Four Series', 'Car_Name_BMW One Series', 'Car_Name_BMW Seven Series', 'Car_Name_BMW Six Series', 'Car_Name_BMW Three Series', 'Car_Name_BMW Two Series', 'Car_Name_BMW X Five', 'Car_Name_BMW X One', 'Car_Name_BMW X Three', 'Car_Name_BYD Atto Three', 'Car_Name_BYD Dolphin', 'Car_Name_BYD Han', 'Car_Name_BYD Seal', 'Car_Name_BYD Tang', 'Car_Name_Changan Alsvin', 'Car_Name_Changan Benni', 'Car_Name_Changan Deepal', 'Car_Name_Changan Eado', 'Car_Name_Changan Hunter', 'Car_Name_Changan Lamore', 'Car_Name_Changan Oshan', 'Car_Name_Changan Raeton', 'Car_Name_Changan Uni K', 'Car_Name_Changan Uni T', 'Car_Name_Chevrolet Aveo', 'Car_Name_Chevrolet Camaro', 'Car_Name_Chevrolet C

In [62]:
import gradio as gr

In [63]:
model = joblib.load("car_price_model.pkl")
features = joblib.load("car_features.pkl")

In [66]:
def predict_price(*inputs):
    data = pd.DataFrame([inputs], columns=features)
    price = model.predict(data)[0]
    return f"Estimated House Price: {price:.2f}"

In [ ]:
import gradio as gr
import joblib
import pandas as pd
import os 
import numpy as np 
MODEL_FILE = "car_price_model.pkl"
FEATURES_FILE = "car_features.pkl"

if not os.path.exists(MODEL_FILE) or not os.path.exists(FEATURES_FILE):
    print(f"FATAL ERROR: Model files not found. Ensure {MODEL_FILE} and {FEATURES_FILE} are in this directory.")
    exit()

try:
    model = joblib.load(MODEL_FILE)
    features_list = joblib.load(FEATURES_FILE)
except Exception as e:
    print(f"Error loading model files: {e}")
    exit()


def predict_car_price(
    price_now, 
    kms_driven, 
    owner_count, 
    purchase_year,
    fuel_type, 
    selling_type, 
    transmission
):
    
    data = {
        'Present_Price': [float(price_now)],
        'Driven_kms': [float(kms_driven)],
        'Owner': [int(owner_count)], 
        'Fuel_Type': [fuel_type],
        'Selling_type': [selling_type],
        'Transmission': [transmission],
    }
 
    df = pd.DataFrame(data, index=[0]) 

    df['Car_Age'] = 2025 - purchase_year
    
 
    df_encoded = pd.get_dummies(df, drop_first=True)
    

    X_input = df_encoded.reindex(columns=features_list, fill_value=0)
    
 
    prediction = model.predict(X_input.values.reshape(1, -1))[0] 
    
    return f"Predicted Price: ₹{prediction:.2f} Lakhs"



inputs = [
    gr.Slider(0.1, 30.0, 0.1, 12.2, label="Present Price (in Lakhs)"), 
    gr.Number(20000, label="Kilometers Driven"),
    gr.Radio([0, 1, 3], 0, label="Number of Previous Owners"),
    gr.Slider(2000, 2024, 1, 2010, label="Year of Purchase"), 
    gr.Radio(['Petrol', 'Diesel', 'CNG'], 'Diesel', label="Fuel Type"),
    gr.Radio(['Dealer', 'Individual'], 'Dealer', label="Selling Type"),
    gr.Radio(['Manual', 'Automatic'], 'Automatic', label="Transmission Type"),
]

interface = gr.Interface(
    fn=predict_car_price,
    inputs=inputs,
    outputs="text",
    title="🚗 Car Price Predictor",
    description="Enter car details to predict the selling price.",
    allow_flagging='never'
)

print("\nLaunching Gradio Interface...")
interface.launch()

C:\Users\Awan\AppData\Roaming\Python\Python313\site-packages\gradio\interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(



Launching Gradio Interface...
* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
